# Cartpole DQN

Deep Q-Learning Network with Keras and OpenAI Gym, based on [Keon Kim's code](https://github.com/keon/deep-q-learning/blob/master/dqn.py).

#### Select processing devices

In [1]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
!pip install keras

  Using cached https://files.pythonhosted.org/packages/5e/10/aa32dad071ce52b5502266b5c659451cfd6ffcbf14e6c8c4f16c0ff5aaab/Keras-2.2.4-py2.py3-none-any.whl


#### Import dependencies

In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os # for creating directories

Using TensorFlow backend.


#### Set parameters

In [2]:
env = gym.make('CartPole-v0') # initialise environment

In [3]:
state_size = env.observation_space.shape[0]
state_size

4

In [6]:
action_size = env.action_space.n
action_size

2

In [4]:
batch_size = 32

In [5]:
n_episodes = 1001 # n games we want agent to play (default 1001)

In [11]:
output_dir = 'model_output/cartpole/'

In [12]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### Define agent

In [7]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # double-ended queue; acts like list, but elements can be added/removed from either end
        self.gamma = 0.95 # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.epsilon = 1.0 # exploration rate: how much to act randomly; more initially than later due to epsilon decay
        self.epsilon_decay = 0.995 # decrease number of random explorations as the agent's performance (hopefully) improves over time
        self.epsilon_min = 0.01 # minimum amount of random exploration permitted
        self.learning_rate = 0.001 # rate at which NN adjusts models parameters via SGD to reduce cost 
        self.model = self._build_model() # private method 
    
    def _build_model(self):
        # neural net to approximate Q-value function:
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu')) # 1st hidden layer; states as input
        model.add(Dense(24, activation='relu')) # 2nd hidden layer
        model.add(Dense(self.action_size, activation='linear')) # 2 actions, so 2 output neurons: 0 and 1 (L/R)
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # list of previous experiences, enabling re-training later

    def act(self, state):
        if np.random.rand() <= self.epsilon: # if acting randomly, take random action
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
      
        return np.argmax(act_values[0]) # pick the action that will give the highest reward (i.e., go left or right?)

    def replay(self, batch_size): # method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory, batch_size) # sample a minibatch from memory
        for state, action, reward, next_state, done in minibatch: # extract data for each minibatch sample
            target = reward # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            if not done: # if not done, then predict future discounted reward
                target = (reward + self.gamma * # (target) = reward + (discount rate gamma) * 
                          np.amax(self.model.predict(next_state)[0])) # (maximum target Q based on future action a')
            target_f = self.model.predict(state) # approximately map current state to future discounted reward
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

#### Interact with environment

In [8]:
agent = DQNAgent(state_size, action_size) # initialise agent

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
done = False
for e in range(n_episodes): # iterate over new episodes of the game
    state = env.reset() # reset state at start of each new episode of the game
    state = np.reshape(state, [1, state_size])
    print(state.shape)
    
    for time in range(200):  # time represents a frame of the game; goal is to keep pole upright as long as possible up to range, e.g., 500 or 5000 timesteps
#         env.render()
        action = agent.act(state)
      # action is either 0 or 1 (move cart left or right); decide on one or other here
        next_state, reward, done, _ = env.step(action) # agent interacts with env, gets feedback; 4 state data points, e.g., pole angle, cart position        
        reward = reward if not done else -10
        print("im next")# reward +1 for each additional frame with pole upright
        print(next_state)
        next_state = np.reshape(next_state, [1, state_size])
        print("next state reshape")
        print(next_state)
        agent.remember(state, action, reward, next_state, done) # remember the previous timestep's state, actions, reward, etc.        
        state = next_state # set "current state" for upcoming iteration to the current next state        
        if done: # episode ends if agent drops pole or we reach timestep 5000
            print("episode: {}/{}, score: {}, e: {:.2}" # print the episode's score and agent's epsilon
                  .format(e, n_episodes, time, agent.epsilon))
            break # exit loop
    if len(agent.memory) > batch_size:
        agent.replay(batch_size) # train the agent by replaying the experiences of the episode
    if e % 50 == 0:
        agent.save(output_dir + "weights_" + '{:04d}'.format(e) + ".hdf5")         

(1, 4)
im next
[ 0.04537882 -0.19463787  0.02155023  0.32202818]
next state reshape
[[ 0.04537882 -0.19463787  0.02155023  0.32202818]]
im next
[ 0.04148606 -0.39005997  0.02799079  0.62142854]
next state reshape
[[ 0.04148606 -0.39005997  0.02799079  0.62142854]]
im next
[ 0.03368486 -0.58556139  0.04041936  0.92279402]
next state reshape
[[ 0.03368486 -0.58556139  0.04041936  0.92279402]]
im next
[ 0.02197364 -0.78120543  0.05887524  1.22790046]
next state reshape
[[ 0.02197364 -0.78120543  0.05887524  1.22790046]]
im next
[ 0.00634953 -0.97703363  0.08343325  1.53843312]
next state reshape
[[ 0.00634953 -0.97703363  0.08343325  1.53843312]]
im next
[-0.01319115 -0.78300886  0.11420191  1.27290927]
next state reshape
[[-0.01319115 -0.78300886  0.11420191  1.27290927]]
im next
[-0.02885132 -0.9793877   0.1396601   1.59906116]
next state reshape
[[-0.02885132 -0.9793877   0.1396601   1.59906116]]
im next
[-0.04843908 -0.78616944  0.17164132  1.35298243]
next state reshape
[[-0.04843908

NameError: name 'output_dir' is not defined

In [13]:
# saved agents can be loaded with agent.load("./path/filename.hdf5") 